In [14]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader
from langchain.schema import SystemMessage

llm = ChatOpenAI(
    temperature=0.1, 
    model="gpt-4o-mini-2024-07-18",
)


class WikipediaUrlSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="URL을 찾기위한 질의입니다. 예시 질의: Ransomware에 대해 조사해줘."
    )


class WikipediaUrlSearchTool(BaseTool):
    name = "WikipediaUrlSearchTool"
    description = """
    질의를 인수로 받아, Wikipedia 검색 결과의 URL 링크 정보를 수집합니다.
    """
    args_schema: Type[
        WikipediaUrlSearchToolArgsSchema
    ] = WikipediaUrlSearchToolArgsSchema

    def _run(self, query) -> Any:
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wiki.run(query)


class DuckDuckGoUrlSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="URL을 찾기위한 질의입니다. 예시 질의: Ransomware에 대해 조사해줘."
    )


class DuckDuckGoUrlSearchTool(BaseTool):
    name = "DuckDuckGoUrlSearchTool"
    description = """
    질의를 인수로 받아, DuckDuckGo 검색 결과의 URL 링크 정보를 수집합니다.
    """
    args_schema: Type[
        DuckDuckGoUrlSearchToolArgsSchema
    ] = DuckDuckGoUrlSearchToolArgsSchema

    def _run(self, query) -> Any:
        ddg = DuckDuckGoSearchRun()
        return ddg.run(query)


class WebResearchToolArgsSchema(BaseModel):
    url: str = Field(
        description="Wikipedia 와 DuckDuckGo에서 찾은 URL 입니다."
    )


class WebResearchTool(BaseTool):
    name = "WebResearchTool"
    description = """
    Wikipedia 와 DuckDuckGo에서 찾은 URL에서 text를 로드해서 문서로 만듭니다.
    문서의 언어는 한글이며, 문서의 마지막에 관련 URL을 제공합니다.
    """
    args_schema: Type[
        WebResearchToolArgsSchema
    ] = WebResearchToolArgsSchema

    def _run(self, url):
        wl = WebBaseLoader([url])
        docs = wl.load()
        text = "\n\n".join([doc.page_content for doc in docs])
        return text


class SaveFileToolArgsSchema(BaseModel):
    text: str


class SaveFileTool(BaseTool):
    name = "SaveFileTool"
    description = """
    WebResearchTool을 통해 생성된 문서를 txt 파일에 저장합니다.
    """
    args_schema: Type[
        SaveFileToolArgsSchema
    ] = SaveFileToolArgsSchema

    def _run(self, text):
        with open('research_doc.txt', 'w', encoding='utf-8') as f:
            f.write(text)
        return "저장 완료"


system_message = SystemMessage(
    content="""
        당신은 조사 전문가입니다.

        사용자의 질의에 대해 Wikipedia 또는 DuckDuckGo 에서 완전하고 정확한 정보를 수집합니다.

        Wikipedia 또는 DuckDuckGo 적절한 웹사이트를 찾으면, 해당 웹사이트의 컨텐츠를 스크랩해야 합니다. 스크랩한 컨텐츠를 사용하여 질문에 대한 자세한 답변을 철저히 조사하고 규격화하세요.

        Wikipedia와 DuckDuckGo 검색 및 찾은 관련 웹사이트의 정보를 결합합니다. 최종 답변이 잘 정리되고 상세하며, 링크(URL)와 함께 자료의 출처가 잘 포함되어 있는지 확인합니다.

        최종 답변은 .txt 파일에 저장해야 하며 역시 모든 출처와 관련 정보를 포함해야 합니다.

        Wikipedia의 정보는 반드시 포함되어야 합니다.

        최종 답변 및 파일 내용은 모두 한글이어야 합니다.
        """
)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaUrlSearchTool(),
        DuckDuckGoUrlSearchTool(),
        WebResearchTool(),
        SaveFileTool(),
    ],
    agent_kwargs={"system_message": system_message},
)

prompt = "XZ backdoor에 대해 조사해줘."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaUrlSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.



Page: XZ Utils
Summary: XZ Utils (previousl

{'input': 'XZ backdoor에 대해 조사해줘.',
 'output': 'XZ 백도어에 대한 조사 결과를 아래와 같이 정리하였습니다.\n\n---\n\n# XZ 백도어에 대한 조사\n\n## 개요\nXZ 백도어는 2024년 2월에 리눅스 유틸리티인 xz의 liblzma 라이브러리에 악성 백도어가 도입된 사건입니다. 이 백도어는 "Jia Tan"이라는 이름을 사용하는 계정에 의해 버전 5.6.0과 5.6.1에 삽입되었습니다. 이 백도어는 특정 Ed448 개인 키를 가진 공격자에게 영향을 받는 리눅스 시스템에서 원격 코드 실행 기능을 제공합니다. 이 문제는 CVE-2024-3094라는 공통 취약점 및 노출 번호가 부여되었으며, CVSS 점수는 10.0으로, 가능한 최고 점수입니다.\n\n## 배경\n이 백도어는 Microsoft 직원이자 PostgreSQL 개발자인 Andres Freund에 의해 발견되었습니다. 그는 Debian Sid에서 성능 저하를 조사하던 중 SSH 연결이 예상보다 높은 CPU 사용량을 발생시키고 Valgrind에서 오류를 일으키는 것을 발견했습니다. Freund는 이 발견을 Openwall 프로젝트의 오픈 소스 보안 메일링 리스트에 보고하여 여러 소프트웨어 공급업체의 주목을 받았습니다.\n\n## 작동 방식\nXZ Utils 소프트웨어 패키지의 5.6.0 및 5.6.1 릴리스에 악성 코드가 포함되어 있습니다. 이 악성 코드는 특정 제3자 SSH 서버 패치가 사용되지 않는 한 비활성 상태입니다. 이 간섭은 악의적인 행위자가 sshd 인증을 우회하고 시스템에 원격으로 무단 접근할 수 있게 할 수 있습니다.\n\n## 대응\n미국 연방 사이버 보안 및 인프라 보안국(CISA)은 영향을 받는 장치가 이전의 안전한 버전으로 롤백할 것을 권장하는 보안 권고를 발표했습니다. 리눅스 소프트웨어 공급업체들은 영향을 받은 패키지를 이전 버전으로 되돌렸습니다. GitHub는 xz 리포지토리의 미러를 비활성화한 후 다시 복원했습니다.\n\n## 결론